<a href="https://colab.research.google.com/github/Amrutha1103/002776773_crashcourse_examples/blob/main/CrashCourse_workedExamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Abstract:

In this notebook we will be seeing the application of gradient boosting algorithm on two different datasets and observe the impact of the GBM on the respective dataset and their features.


I have used "Income_Evalutation" and "Shark_tank_india" dataset.

In [2]:
# Importing all required libraries

!pip install shap
!pip install lime
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install h2o




import pandas as pd
import numpy as np
import shap
import lime
from lime import lime_tabular
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import random
from sklearn.ensemble import RandomForestRegressor
from matplotlib.pyplot import figure
from sklearn.tree import export_graphviz
import pydot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns



from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

plt.rcParams["figure.figsize"] = (32, 24)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 6.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283859 sha256=c5f9a9889c336435555d38fa7d7b21aa61d1b8a8acb112a425fdc2a78f8f8f0f
  Stored in directory: /root/.cache/pip/wheels/ed/d7/c9/5a0130d06d6310bc6cbe55220e6e72dcb8c4eff9a478717066
Successfully built lime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: ht

## "Income_Evaluation" Dataset:

Prediction task is to determine whether a person makes over 50K a year.

Listing of attributes: 50K, <=50K.

### Data description:

age: continuous.

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands

In [3]:
# To read CSV file into Pandas DataFrame
data_df = pd.read_csv(
    "https://raw.githubusercontent.com/Amrutha1103/002776773_crashcourse_examples/main/income_evaluation.csv"
) 
data_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
# finding if dataset has missing values or null values as input
data_df.isnull().sum()

age                0
 workclass         0
 fnlwgt            0
 education         0
 education-num     0
 marital-status    0
 occupation        0
 relationship      0
 race              0
 sex               0
 capital-gain      0
 capital-loss      0
 hours-per-week    0
 native-country    0
 income            0
dtype: int64

In [5]:
#to display information of dataframe "df"
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              32561 non-null  int64 
 1    workclass       32561 non-null  object
 2    fnlwgt          32561 non-null  int64 
 3    education       32561 non-null  object
 4    education-num   32561 non-null  int64 
 5    marital-status  32561 non-null  object
 6    occupation      32561 non-null  object
 7    relationship    32561 non-null  object
 8    race            32561 non-null  object
 9    sex             32561 non-null  object
 10   capital-gain    32561 non-null  int64 
 11   capital-loss    32561 non-null  int64 
 12   hours-per-week  32561 non-null  int64 
 13   native-country  32561 non-null  object
 14   income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [6]:
#checking for the shape of the dataframe
data_df.shape

(32561, 15)

In [7]:
#To display the columns of the datasets
data_df.columns

Index(['age', ' workclass', ' fnlwgt', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' income'],
      dtype='object')

In [8]:
np.unique(data_df[' income'])

array([' <=50K', ' >50K'], dtype=object)

In [9]:
np.unique(data_df[' marital-status'])

array([' Divorced', ' Married-AF-spouse', ' Married-civ-spouse',
       ' Married-spouse-absent', ' Never-married', ' Separated',
       ' Widowed'], dtype=object)

In [10]:
data_df.drop(columns=' fnlwgt',inplace=True)

In [11]:
data_df.columns

Index(['age', ' workclass', ' education', ' education-num', ' marital-status',
       ' occupation', ' relationship', ' race', ' sex', ' capital-gain',
       ' capital-loss', ' hours-per-week', ' native-country', ' income'],
      dtype='object')

In [12]:
col_names = ['age', 'workclass', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
             'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

data_df.columns = col_names

data_df.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'sex', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income'],
      dtype='object')

In [13]:
#checking for null values
data_df.isnull().sum()

age               0
workclass         0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64

In [14]:
# adding new the categorical column
bins = [16, 24, 64, 90]
labels = ["young","adult","old"]
data_df['age_types'] = pd.cut(data_df['age'], bins=bins, labels=labels)
data_df['income_num'] = np.where(data_df['income'] == ' >50K', 1, 0).astype('int16')

In [15]:
data_df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,age_types,income_num
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,adult,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,adult,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,adult,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,adult,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,adult,0


In [16]:
np.unique(data_df.education)

array([' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th',
       ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate',
       ' HS-grad', ' Masters', ' Preschool', ' Prof-school',
       ' Some-college'], dtype=object)

In [17]:
data_df.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'sex', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income',
       'age_types', 'income_num'],
      dtype='object')

In [18]:
strip_columns = ['workclass', 'education','marital_status',
       'occupation', 'relationship', 'race', 'sex','native_country', 'income']

In [19]:
for i in strip_columns:
    data_df[i] = data_df[i].str.strip()
np.unique(data_df.workclass)

array(['?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private',
       'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'],
      dtype=object)

In [21]:
np.unique(data_df.workclass)

array(['?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private',
       'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'],
      dtype=object)

In [22]:
from sklearn.preprocessing import LabelEncoder

In [25]:
def label_encoder(a):
    le = LabelEncoder()
    data_df[a] = le.fit_transform(data_df[a])

In [26]:
data_df.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'sex', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income',
       'age_types', 'income_num'],
      dtype='object')

In [27]:
label_list = ['workclass', 'education','marital_status',
       'occupation', 'relationship', 'race', 'sex','native_country', 'income']
for i in label_list:
    label_encoder(i)

In [29]:
data_df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,age_types,income_num
0,39,7,9,13,4,1,1,4,1,2174,0,40,39,0,adult,0
1,50,6,9,13,2,4,0,4,1,0,0,13,39,0,adult,0
2,38,4,11,9,0,6,1,4,1,0,0,40,39,0,adult,0
3,53,4,1,7,2,6,0,2,1,0,0,40,39,0,adult,0
4,28,4,9,13,2,10,5,2,0,0,0,40,5,0,adult,0


In [31]:
#this code initializes a MinMaxScaler object and loads some data into a pandas DataFrame. 
#It does not perform any data normalization yet, but the scaler object can be used to perform normalization on the DataFrame later.


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,age_types,income_num
0,39,7,9,13,4,1,1,4,1,2174,0,40,39,0,adult,0
1,50,6,9,13,2,4,0,4,1,0,0,13,39,0,adult,0
2,38,4,11,9,0,6,1,4,1,0,0,40,39,0,adult,0
3,53,4,1,7,2,6,0,2,1,0,0,40,39,0,adult,0
4,28,4,9,13,2,10,5,2,0,0,0,40,5,0,adult,0


In [33]:
#purpose of the below code may be to prepare the data for a machine learning model, since scaling can be an important preprocessing step for many models.
scaler.fit(data_df.drop(['income','age_types','income_num'],axis=1))

MinMaxScaler()

In [34]:
scaled_features = scaler.transform(data_df.drop(['income','age_types','income_num'],axis=1))

In [35]:
data_df.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'sex', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income',
       'age_types', 'income_num'],
      dtype='object')

In [36]:
columns=['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'sex', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country']

In [37]:
df_scaled = pd.DataFrame(scaled_features,columns=columns)
df_scaled.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,0.301370,0.875,0.600000,0.800000,0.666667,0.071429,0.2,1.0,1.0,0.02174,0.0,0.397959,0.951220
1,0.452055,0.750,0.600000,0.800000,0.333333,0.285714,0.0,1.0,1.0,0.00000,0.0,0.122449,0.951220
2,0.287671,0.500,0.733333,0.533333,0.000000,0.428571,0.2,1.0,1.0,0.00000,0.0,0.397959,0.951220
3,0.493151,0.500,0.066667,0.400000,0.333333,0.428571,0.0,0.5,1.0,0.00000,0.0,0.397959,0.951220
4,0.150685,0.500,0.600000,0.800000,0.333333,0.714286,1.0,0.5,0.0,0.00000,0.0,0.397959,0.121951


#### Train_test_split

In [39]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

In [41]:
X = df_scaled
y= data_df.income

In [42]:
# Implementing Oversampling for Handling Imbalanced 
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_resample(X,y)

In [43]:
X_res.shape,y_res.shape

((48166, 13), (48166,))

In [44]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(y)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({0: 24720, 1: 7841})
Resampled dataset shape Counter({0: 24083, 1: 24083})


In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,test_size=0.20,random_state=101,shuffle=True)
print("X_train shape:",X_train.shape)
print("X_test shape:",X_test.shape)
print("y_train shape:",y_train.shape)
print("y_test shape:",y_test.shape)

X_train shape: (38532, 13)
X_test shape: (9634, 13)
y_train shape: (38532,)
y_test shape: (9634,)


In [46]:
# To store results of models
result_dict_train = {}
result_dict_test = {}

In [71]:
#Define Gradient Boosting classifier with hyperparameters

gbc = GradientBoostingClassifier(n_estimators= 500, learning_rate=0.05, random_state=100, max_features = 5)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [72]:
#To get the report of number of classifications and misclassifications, we are using confusion matrix to get number of correct and incorrect classifications

print(confusion_matrix(y_test, gbc.predict(X_test)))

[[4148  661]
 [ 476 4349]]


### Gradient Boosting of Income_evaluation dataset

In [47]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


In [48]:
#below code uses Gradient Boosting to build a classification model and evaluates its performance using cross-validation

gradient_booster = GradientBoostingClassifier(learning_rate=0.1)
accuracies = cross_val_score(gradient_booster, X_train, y_train, cv=5)
gradient_booster.fit(X_train,y_train)

print("Train Score:",np.mean(accuracies))
print("Test Score:",gradient_booster.score(X_test,y_test))

Train Score: 0.8650730755956099
Test Score: 0.865476437616774


In [49]:
result_dict_train["Gradient-Boost Default Train Score"] = np.mean(accuracies)
result_dict_test["Gradient-Boost Default Test Score"] = gradient_booster.score(X_test,y_test)

In [50]:
grid = {
    'learning_rate':[0.01,0.05,0.1],
    'n_estimators':np.arange(100,500,100),
}

gb = GradientBoostingClassifier()
gb_cv = GridSearchCV(gb, grid, cv = 4)
gb_cv.fit(X_train,y_train)
print("Best Parameters:",gb_cv.best_params_)
print("Train Score:",gb_cv.best_score_)
print("Test Score:",gb_cv.score(X_test,y_test))

Best Parameters: {'learning_rate': 0.1, 'n_estimators': 400}
Train Score: 0.8960344648603757
Test Score: 0.9013909072036538


In [51]:
result_dict_train["Gradient-Boost GridSearch Train Score"] = gb_cv.best_score_
result_dict_test["Gradient-Boost GridSearch Test Score"] = gb_cv.score(X_test,y_test)

In [78]:
#To check the accuracy of the model

print("GBC accuracy is %2.2f" %accuracy_score(y_test, gbc.predict(X_test)))

GBC accuracy is 0.88


We have got the accuarcy as 89% for train set and 90% for test set

## Sharktank India Dataset

###Data description:

*   Episode_number - Number of the episode 
*   Pitch_number - Number of the Pitch
*   Brand_name - Name of the brand Idea
*   Idea - behind the brand building Deal
*   Deal - done or not ; 1 - YES, 0 - NO
*   Pitcher_ask_amount - Amount asked by the pitchers
*   Ask_equity - Equity offered by the pitchers
*   Ask_valuation - Valuation asked by pitchers
*   Deal_amount - Final Deal Amount
*   Deal_equity - Final Deal equity percentage
*   Deal_valuation - Final Valuation of Company after Deal
*   Ashneer_present - Ashneer was present during the pitching ; 1 - YES, 0 - NO
*   Anupam_present - Anupam was present during the pitching ; 1 - YES, 0 - NO
*   Aman_present - Aman was present during the pitching ; 1 - YES, 0 - NO
*   Namita_present - Namita was present during the pitching ; 1 - YES, 0 - NO
*   Vineeta_present - Vineeta was present during the pitching ; 1 - YES, 0 - NO
*   Peyush_present - Peyush was present during the pitching ; 1 - YES, 0 - NO
*   Ghazal_present - Ghazal was present during the pitching ; 1 - YES, 0 - NO
*   Total_sharks_invested - Number of total sharks invested in the Company
*   Amount_per_shark - Amount per shark invested
*   Equity_per_shark - Final Equity gained per Shark


In [80]:
# To read CSV file into Pandas DataFrame
df2 = pd.read_csv('https://raw.githubusercontent.com/Amrutha1103/002776773_crashcourse_examples/main/Shark%20Tank%20India%20Dataset.csv')
df2.head()

,episode_number,pitch_number,brand_name,idea,deal,pitcher_ask_amount,ask_equity,ask_valuation,deal_amount,deal_equity,...,ashneer_deal,anupam_deal,aman_deal,namita_deal,vineeta_deal,peyush_deal,ghazal_deal,total_sharks_invested,amount_per_shark,equity_per_shark
0,1,1,BluePine Industries,Frozen Momos,1,50.0,5.0,1000.00,75.0,16.00,...,1,0,1,0,1,0,0,3,25.0,5.333333
1,1,2,Booz scooters,Renting e-bike for mobility in private spaces,1,40.0,15.0,266.67,40.0,50.00,...,1,0,0,0,1,0,0,2,20.0,25.000000
2,1,3,Heart up my Sleeves,Detachable Sleeves,1,25.0,10.0,250.00,25.0,30.00,...,0,1,0,0,1,0,0,2,12.5,15.000000
3,2,4,Tagz Foods,Healthy Potato Chips,1,70.0,1.0,7000.00,70.0,2.75,...,1,0,0,0,0,0,0,1,70.0,2.750000
4,2,5,Head and Heart,Brain Development Course,0,50.0,5.0,1000.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0.0,0.000000


In [56]:
X1 = df2.drop(["brand_name","idea"], axis=1)  # Training Feature which conatins all the remaining columns of the data_df
Y1 = df2["deal"] #Target variable we want to predict

### Test_train_split

In [57]:
#splitting the data into the test and training datasets using "train_test_split" function and  "sklearn.model_selection" module
(
    X1_train,
    X1_test,
    Y1_train,
    Y1_test,
) = train_test_split( 
    X1, Y1, test_size=0.1, random_state=42 
) # keeping test size 0.1 i.e 10% of the data will be used for the testing and the remaining for the training
X1_train_100 = shap.utils.sample(
    X1_train, 100
)  # Taking 100 samples out for SHAP analysis as it is a computationally expensive process
X1_test_100 = shap.utils.sample(
    X1_test, 100
) 

In [60]:
# Implementing Oversampling for Handling Imbalanced 
smk = SMOTETomek(random_state=42)
X1_res,Y1_res=smk.fit_resample(X1,Y1)

In [62]:
X1_res.shape,Y1_res.shape

((120, 26), (120,))

In [63]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y1)))
print('Resampled dataset shape {}'.format(Counter(Y1_res)))

Original dataset shape Counter({1: 65, 0: 52})
Resampled dataset shape Counter({1: 60, 0: 60})


In [64]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X_res,y_res,test_size=0.20,random_state=101,shuffle=True)
print("X1_train shape:",X1_train.shape)
print("X1_test shape:",X1_test.shape)
print("y1_train shape:",Y1_train.shape)
print("y1_test shape:",Y1_test.shape)

X1_train shape: (38532, 13)
X1_test shape: (9634, 13)
y1_train shape: (38532,)
y1_test shape: (9634,)


In [65]:
# To store results of models
result_dict_train = {}
result_dict_test = {}

In [66]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [67]:
gradient_booster = GradientBoostingClassifier(learning_rate=0.1)
accuracies = cross_val_score(gradient_booster, X1_train, Y1_train, cv=5)
gradient_booster.fit(X1_train,Y1_train)

print("Train Score:",np.mean(accuracies))
print("Test Score:",gradient_booster.score(X_test,y_test))

Train Score: 0.8650730755956099
Test Score: 0.865476437616774


In [68]:
result_dict_train["Gradient-Boost Default Train Score"] = np.mean(accuracies)
result_dict_test["Gradient-Boost Default Test Score"] = gradient_booster.score(X1_test,Y1_test)

In [69]:
grid = {
    'learning_rate':[0.01,0.05,0.1],
    'n_estimators':np.arange(100,500,100),
}

gb = GradientBoostingClassifier()
gb_cv = GridSearchCV(gb, grid, cv = 4)
gb_cv.fit(X1_train,Y1_train)
print("Best Parameters:",gb_cv.best_params_)
print("Train Score:",gb_cv.best_score_)
print("Test Score:",gb_cv.score(X1_test,Y1_test))

Best Parameters: {'learning_rate': 0.1, 'n_estimators': 400}
Train Score: 0.8960863697705802
Test Score: 0.9013909072036538


In [70]:
result_dict_train["Gradient-Boost GridSearch Train Score"] = gb_cv.best_score_
result_dict_test["Gradient-Boost GridSearch Test Score"] = gb_cv.score(X1_test,Y1_test)

In [73]:
#Define Gradient Boosting classifier with hyperparameters

gbc = GradientBoostingClassifier(n_estimators= 500, learning_rate=0.05, random_state=100, max_features = 5)
gbc.fit(X1_train, Y1_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [74]:
#To get the report of number of classifications and misclassifications, we are using confusion matrix to get number of correct and incorrect classifications

print(confusion_matrix(Y1_test, gbc.predict(X1_test)))

[[4148  661]
 [ 476 4349]]


In [75]:
#To check the accuracy of the model

print("GBC accuracy is %2.2f" %accuracy_score(Y1_test, gbc.predict(X1_test)))

GBC accuracy is 0.88


From the above analysis we have got accuracy for sharktank dataset as 88%

In [76]:
#To get the classification report

from sklearn.metrics import classification_report

pred=gbc.predict(X1_test)

print(classification_report(Y1_test, pred))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      4809
           1       0.87      0.90      0.88      4825

    accuracy                           0.88      9634
   macro avg       0.88      0.88      0.88      9634
weighted avg       0.88      0.88      0.88      9634



### Hyperparameter tuning

In [77]:
from sklearn.model_selection import GridSearchCV

grid = {

    'learning_rate':[0.01,0.05,0.1],

    'n_estimators':np.arange(100,500,100),

}


gb = GradientBoostingClassifier()

gb_cv = GridSearchCV(gb, grid, cv = 4)

gb_cv.fit(X_train,Y1_train)

print("Best Parameters:",gb_cv.best_params_)

print("Train Score:",gb_cv.best_score_)

print("Test Score:",gb_cv.score(X1_test,Y1_test))

Best Parameters: {'learning_rate': 0.1, 'n_estimators': 400}
Train Score: 0.8960604173154781
Test Score: 0.9013909072036538


From the hyper parameter tuning we have got train set accuracy as 89% and test set accuracy as 90%

#### Conclusion:

Gradient Boosting is a powerful machine learning algorithm that has been widely used for classification tasks. It works by combining multiple weak classifiers into a strong classifier in a sequential manner, where each subsequent classifier is trained to correct the mistakes made by the previous one.

Overall, gradient boosting has several advantages over other classification algorithms, including its ability to handle complex datasets, handle missing data, and avoid overfitting. However, it can also be computationally intensive and may require careful tuning of hyperparameters to achieve optimal performance.

In conclusion, Gradient Boosting is a highly effective algorithm for classification tasks, and its performance can be further improved by careful selection of hyperparameters and appropriate data preprocessing.

From the above two examples we can see the GBC accuarcy for income_evaluation is 89% and the accuracy for shark_tabk_india as 88%



#### References

H2O AI AutoML - https://docs.h2o.ai/

https://machinelearningmastery.com/gradient-boosting-machine-ensemble-in-python/

https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/


#### Licences:

MIT License

Copyright (c) 2023 Amrutha Sai Edara

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE